In [1]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 26.0 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 


In [36]:
import numpy as np
import pandas as pd
import random
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate, utils, agent
from kaggle_environments.helpers import Observation, Configuration

from typing import Dict, List, Tuple

###Вспомогательная функция для выполнения хода в общем случае



In [47]:
def default_move(conf: Configuration) -> int:
  return random.randrange(0, conf.signs)

###Описание всех агентов

In [48]:
#Копирует предыдущий ход оппонента
def copy_agent(obs: Observation, conf: Configuration) -> int:
  if obs.step > 0:
    return obs.lastOpponentAction
  else:
    return default_move(conf)

#Выбирает ход сильнее, чем предыдущий оппонента
def inc_agent(obs: Observation, conf: Configuration) -> int:
  if obs.step > 0:
    return obs.lastOpponentAction + 1 if obs.lastOpponentAction < 2 else 0
  else:
    return default_move(conf)

#Выбирает ход слабее, чем предыдущий оппонента
def dec_agent(obs: Observation, conf: Configuration) -> int:
  if obs.step > 0:
    return obs.lastOpponentAction - 1 if obs.lastOpponentAction > 0 else 2
  else:
    return default_move(conf)

#Если оппонент в предыдущий ход выбрал камень - выбирает камень,
#иначе бумагу или ножницы
def if_rock_agent(obs: Observation, conf: Configuration) -> int:
  if obs.step > 0:
    return (0 if obs.lastOpponentAction == 0
            else random.choice([1, 2]))
  else:
    return default_move(conf)

#Если оппонент в предыдущий ход выбрал бумагу - выбирает бумагу,
#иначе камень или ножницы
def if_paper_agent(obs: Observation, conf: Configuration) -> int:
  if obs.step > 0:
    return (1 if obs.lastOpponentAction == 1
            else random.choice([0, 2]))
  else:
    return default_move(conf)

#Если оппонент в предыдущий ход выбрал ножницы - выбирает ножницы,
#иначе камень или бумагу
def if_scissors_agent(obs: Observation, conf: Configuration) -> int:
  if obs.step > 0:
    return (2 if obs.lastOpponentAction == 2
            else random.choice([0, 1]))
  else:
    return default_move(conf)

#Делает случайный ход
def random_agent(obs: Observation, conf: Configuration) -> int:
  return random.randrange(random.randrange(0,2), conf.signs)

#Выбирает камень
def rock_agent(obs: Observation, conf: Configuration) -> int:
  return 0

#Выбирает не камень
def not_rock_agent(obs: Observation, conf: Configuration) -> int:
  return random.randrange(1, conf.signs)

#Выбирает бумагу
def paper_agent(obs: Observation, conf: Configuration) -> int:
  return 1

#Выбирает не бумагу
def not_paper_agent(obs: Observation, conf: Configuration) -> int:
  return random.randrange(0, conf.signs, 2)

#Выбирает ножницы
def scissors_agent(obs: Observation, conf: Configuration) -> int:
  return 2

#Выбирает не ножницы
def not_scissors_agent(obs: Observation, conf: Configuration) -> int:
  return random.choice([0, 1])

###Наполнение словаря агентов

In [32]:
all_agents: Dict[str, object] = {
    "rock_agent": rock_agent,
    "not_rock_agent": not_rock_agent,
    "paper_agent": paper_agent,
    "not_paper_agent": not_paper_agent,
    "scissors_agent": scissors_agent,
    "not_scissors_agent": not_scissors_agent,
    "copy_agent": copy_agent,
    "random_agent": random_agent,
    "inc_agent": inc_agent,
    "dec_agent": dec_agent,
    "if_rock_agent": if_rock_agent,
    "if_paper_agent": if_paper_agent,
    "if_scissors_agent": if_scissors_agent
}

###Турнир каждый с каждым

In [49]:
scores: Dict[str, int] = {name: 0 for name in all_agents.keys()}

#Проведение турнира, где каждый агент играет с каждым
for round in itertools.product(all_agents.items(), all_agents.items()):
  player_1 = round[0]
  player_2 = round[1]
  if player_1[0] != player_2[0]:
    result = evaluate("rps",
                      [player_1[1], player_2[1]],
                      configuration={"episodeSteps": 99})
    scores[player_1[0]] = (scores[player_1[0]] + 1 if result[0][0] > -1
                           else scores[player_1[0]])
    scores[player_2[0]] = (scores[player_2[0]] + 1 if result[0][1] > -1
                           else scores[player_2[0]])
  print(scores)

#Ранжирование агентов
placement: List[Tuple[str, int]] = sorted(list(scores.items()), 
                                          key=lambda score: score[1],
                                          reverse=True)

print(f"WINNER: {placement[0][0]}")

{'rock_agent': 0, 'not_rock_agent': 0, 'paper_agent': 0, 'not_paper_agent': 0, 'scissors_agent': 0, 'not_scissors_agent': 0, 'copy_agent': 0, 'random_agent': 0, 'contr_agent': 0}
{'rock_agent': 1, 'not_rock_agent': 1, 'paper_agent': 0, 'not_paper_agent': 0, 'scissors_agent': 0, 'not_scissors_agent': 0, 'copy_agent': 0, 'random_agent': 0, 'contr_agent': 0}
{'rock_agent': 1, 'not_rock_agent': 1, 'paper_agent': 1, 'not_paper_agent': 0, 'scissors_agent': 0, 'not_scissors_agent': 0, 'copy_agent': 0, 'random_agent': 0, 'contr_agent': 0}
{'rock_agent': 2, 'not_rock_agent': 1, 'paper_agent': 1, 'not_paper_agent': 0, 'scissors_agent': 0, 'not_scissors_agent': 0, 'copy_agent': 0, 'random_agent': 0, 'contr_agent': 0}
{'rock_agent': 3, 'not_rock_agent': 1, 'paper_agent': 1, 'not_paper_agent': 0, 'scissors_agent': 0, 'not_scissors_agent': 0, 'copy_agent': 0, 'random_agent': 0, 'contr_agent': 0}
{'rock_agent': 3, 'not_rock_agent': 1, 'paper_agent': 1, 'not_paper_agent': 0, 'scissors_agent': 0, 'not_